## Library
---

In [1]:
import os
import time
import numpy as np
from ase import Atoms, neighborlist, Atom
from ase.io import write
from ase.lattice.cubic import FaceCenteredCubic
from ase.lattice.hexagonal import HexagonalClosedPacked

## Algorithm
---

In [2]:
'''
site energy -> activation energy -> diffusion rate

site energy(E_i) = -N*(E_b)/2

activation energy(E_a) = E_0 + alpha * E_r
E_r = E_i(end) - E_i(start)

diffusion rate = f*exp(-E_a/(k*T)) (Arrhenius)
f is ~ 10^13 for most metals
'''

def get_site_energy(bond_energy, bond_num):
    return -bond_num * bond_energy/2

def get_activation_energy(e_start, e_end, alpha = 0.1, e0 = 0):
    e_reaction = e_end-e_start
    if e_reaction>=0:
        return e0 + (1+alpha)*e_reaction
    else:
        return e0 + alpha*e_reaction

def get_diffusion_rate(e_a, T=300, f=1E13):
    k_B = 8.617333262145e-5  # Boltzmann constant in eV/K
    return f*np.exp(-e_a/(k_B*T))

In [28]:
# function to create fcc lattice
def create_fcc_lattice(symbol, lattice_constant, repetitions):
    """
    Create an FCC lattice of atoms.

    Args:
    - symbol: The chemical symbol of the atoms.
    - lattice_constant: The lattice constant of the FCC lattice.
    - repetitions: The number of repetitions of the unit cell in each direction.
                [x, y, z]
    - vacuum: The thickness of the vacuum layer outside the plane perpendicular to the z-axis.

    Returns:
    - An Atoms object representing the FCC lattice.
    """

    # Define the positions of the atoms in the unit cell
    positions = [[0, 0, 0],
                 [0.5, 0.5, 0],
                 [0.5, 0, 0.5],
                 [0, 0.5, 0.5]]

    # Scale the positions by the lattice constant
    positions = [[x * lattice_constant for x in pos] for pos in positions]

    # Create the FCC lattice
    fcc_lattice = Atoms([symbol] * 4, positions=positions, cell=[lattice_constant] * 3, pbc=[True, True, False])

    # Repeat the unit cell in x and y directions
    fcc_lattice *= repetitions

    return fcc_lattice

# function to create bcc lattice
def create_bcc_lattice(symbol, lattice_constant, repetitions):
    """
    Create a BCC lattice of atoms.

    Args:
    - symbol: The chemical symbol of the atoms.
    - lattice_constant: The lattice constant of the BCC lattice.
    - repetitions: The number of repetitions of the unit cell in each direction.
                [x, y, z]

    Returns:
    - An Atoms object representing the BCC lattice.
    """

    # Define the positions of the atoms in the unit cell
    positions = [[0, 0, 0],
                 [0.5, 0.5, 0.5]]

    # Scale the positions by the lattice constant
    positions = [[x * lattice_constant for x in pos] for pos in positions]

    # Create the BCC lattice
    bcc_lattice = Atoms([symbol] * 2, positions=positions, cell=[lattice_constant] * 3, pbc=[True, True, False])

    # Repeat the unit cell in x, y, and z directions
    bcc_lattice *= repetitions

    return bcc_lattice


# function to create hcp lattice
def create_hcp_lattice(symbol, lattice_constant, repetitions):
    """
    Create a hexagonal close-packed (HCP) lattice of atoms.

    Args:
    - symbol: The chemical symbol of the atoms.
    - lattice_constant: The lattice constant of the HCP lattice.
    - repetitions: The number of repetitions of the unit cell in each direction.
                [x, y, z]

    Returns:
    - An Atoms object representing the HCP lattice.
    """

    # Define the positions of the atoms in the unit cell
    positions = [[0, 0, 0],
                 [2/3, 1/3, 1/2]]

    # Scale the positions by the lattice constant
    positions = [[x * lattice_constant for x in pos] for pos in positions]

    # Create the HCP lattice
    hcp_lattice = Atoms([symbol] * 2, positions=positions, cell=[lattice_constant] * 3, pbc=[True, True, False])

    # Repeat the unit cell in x, y, and z directions
    hcp_lattice *= repetitions

    return hcp_lattice


In [4]:
# function to find the closest neighbors of a vacancy

def get_closest_neighbors(lattice, vacancy_index):
    """
    Get the indices of the closest neighboring atoms to a vacancy in a lattice.

    Args:
    - lattice: The lattice containing the vacancy and neighboring atoms.
    - vacancy_index: The index of the vacancy in the lattice.
    - num_neighbors: The number of neighboring atoms to return.

    Returns:
    - A list of the indices of the closest neighboring atoms to the vacancy.
    """
    # Create a neighbor list
    cutoff = neighborlist.natural_cutoffs(lattice)
    nl = neighborlist.NeighborList(cutoff, self_interaction=False, bothways=True)
    nl.update(lattice)

    # Get the indices of the atoms that are neighbors to the vacancy
    indices, offsets = nl.get_neighbors(vacancy_index)

    return indices



In [9]:
# function to diffusion of a vacancy

def find_candidate(lattice):
    global diffusion_rate

    candidate_table = []    # list of candidate atoms. element: [start_pos, end_pos]
    diffusion_table = []    # list of diffusion rate. Indices corresponding to the candidate_table  element: diffusion rate

    # 1. find a candididate
    vacancy_indices = lattice.symbols.search('X')
    for vac_idx in vacancy_indices:
        lattice[vac_idx].symbol = 'Cu'
        candidate_idx = get_closest_neighbors(lattice, vac_idx)
        lattice[vac_idx].symbol = 'X'
        end_point_bond_num = len(candidate_idx)-1

        # 2. modify event table
        for idx in candidate_idx:
            # position
            candidate_table.append([idx, vac_idx])

            # diffusion rate
            start_point_bond_num = len(get_closest_neighbors(lattice, idx))
            diffusion_table.append(diffusion_rate[start_point_bond_num][end_point_bond_num])
    
    return candidate_table, diffusion_table

def diffuse_one_step(lattice):
    global time_elapsed

    # 1. find a candidate
    candidate_table, diffusion_table = find_candidate(lattice)
    diffusion_table = np.array(diffusion_table)
    total_diffusion_rate = diffusion_table.sum()

    # 2. pick random numbers
    u = np.random.uniform(low=1e-6, high=1)
    u_time = np.random.uniform(low=1e-6, high=1)
    cum_dif = np.cumsum(diffusion_table)
    chosen_idx = np.argwhere(u*total_diffusion_rate < cum_dif)[0][0]
    # print(chosen_idx)

    # 3. time update
    delta_t = -np.log(u_time)/total_diffusion_rate
    time_elapsed += delta_t

    # 4. diffusion(swap the position of atom 'A' and vacancy 'X')
    start_idx = candidate_table[chosen_idx][0]
    end_idx = candidate_table[chosen_idx][1]
    start_pos = lattice[start_idx].position.copy()
    end_pos = lattice[end_idx].position
    vacancy_z = end_pos[2]
    
    lattice[start_idx].position = end_pos
    lattice[end_idx].position = start_pos
    
    return vacancy_z

## Parameter setting
---

In [6]:
# parameter for diffusion rate
'''
bond energy : 200kJ/mol ~~ 2.07 eV/particle
fcc -> 12 nearest neighbors
'''
bond_energy = 2.07 
temperature = 500
e0 = 0.1
num_closest_neighbors = 12

# site energy, e_(bond number)
e_site = np.zeros(num_closest_neighbors)
for i in range(num_closest_neighbors):
    e_site[i] = get_site_energy(bond_energy, i)

# activation energy, e_a_(start to end)
e_a = np.zeros((num_closest_neighbors, num_closest_neighbors))
for i in range(num_closest_neighbors):
    for j in range(num_closest_neighbors):
        e_a[i, j] = get_activation_energy(e_site[i], e_site[j], e0=e0)

# diffusion rate, rate_(start to end)
diffusion_rate = np.zeros((num_closest_neighbors, num_closest_neighbors))
for i in range(num_closest_neighbors):
    for j in range(num_closest_neighbors):
        diffusion_rate[i, j] = get_diffusion_rate(e_a[i, j], temperature)
   

In [7]:
# value check
# print(e_site)
# print('-----------------')
# # print(e_a)
# print('-----------------')
# print(diffusion_rate)

## Simulation
---

In [29]:
# parameter tuning
time_elapsed = 0
save_file_name = 'ovito/test_hcp_single.xyz'
steps = 1000
width = 10
depth = 10
height = 3
vacancy_idx = 100

# atom_type = 'Cu'
# lattice_param = 3.6
# atom_type = 'Fe'
# lattice_param = 2.87
atom_type = 'Ti'
lattice_param = 2.94

# create lattice
# lattice = create_fcc_lattice(atom_type, lattice_param, [width, depth, height])
# lattice = create_bcc_lattice(atom_type, lattice_param, [width, depth, height])
lattice = create_hcp_lattice(atom_type, lattice_param, [width, depth, height])

# make vacancy
lattice[vacancy_idx].symbol = 'X'
prev_vacncy_z = None
vacancy_z = lattice[vacancy_idx].position[2]
write(save_file_name, lattice)



/tmp/ipykernel_843/2467453919.py:91: DeprecationWarning: `product` is deprecated as of NumPy 1.25.0, and will be removed in NumPy 2.0. Please use `prod` instead.
  hcp_lattice *= repetitions


In [ ]:
# start simulation
for i in range(1, steps+1):
    prev_vacncy_z = vacancy_z
    vacancy_z = diffuse_one_step(lattice)
    # print(prev_vacncy_z, vacancy_z)
    if prev_vacncy_z != vacancy_z:
        write(save_file_name, lattice, append=True)
    elif i % 10 == 0:
        write(save_file_name, lattice, append=True)
        print(f'---------------- step {i} ---------------------')
        print(f'Time elapsed in simul: {time_elapsed} s')

## Test area
---

In [24]:
for idx in get_closest_neighbors(lattice, 1):
    lattice[idx].symbol = 'Y'

write('bcc_test2.xyz', lattice)

In [30]:
get_closest_neighbors(lattice, 1)

array([  7,   6,  66,  61,  60,   8,   2,   3,  62,  68,   0,  55, 541])